## 最終課題

In [ ]:
import time
import random
from collections import deque
from urllib.parse import urljoin, urlparse, urldefrag

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup, Comment
from urllib import robotparser

# 開始URL
START_URL = "https://www.musashino-u.ac.jp/"
DOMAIN = urlparse(START_URL).netloc  # 例: "www.musashino-u.ac.jp"

# 除外する拡張子（PDFや画像など）
EXCLUDE_EXTS = {".pdf", ".jpg", ".jpeg", ".png", ".gif", ".webp", ".svg", ".zip"}

# クロール制御パラメータ
CONNECT_READ_TIMEOUT = (5, 30)  # 接続5秒、読み取り30秒
SLEEP_BETWEEN_REQUESTS = 1.0     # アクセス間隔（秒）
RESPECT_ROBOTS = True             # robots.txt を尊重するか
MAX_PAGES = 2000                  # 取得ページ数の上限

def make_session() -> requests.Session:
    """
    リトライ設定付きの requests.Session を作成。
    - 総リトライ8回、接続/読み取り各5回
    - backoff_factor=2.0（指数的に待機時間増加）
    - 429/5xx を対象
    """
    retry = Retry(
        total=8,
        connect=5,
        read=5,
        backoff_factor=2.0,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=frozenset(["GET", "HEAD"]),
        raise_on_status=False,
        respect_retry_after_header=True,
    )
    adapter = HTTPAdapter(max_retries=retry, pool_connections=5, pool_maxsize=5)
    s = requests.Session()
    s.headers.update({
        "User-Agent": (
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/124.0.0.0 Safari/537.36"
        ),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "ja,en;q=0.8",
        "Connection": "keep-alive",
    })
    s.mount("https://", adapter)
    s.mount("http://", adapter)
    return s

# 礼儀的な待機（固定秒＋ジッター）
def polite_sleep():
    time.sleep(SLEEP_BETWEEN_REQUESTS)

# URLが同一ドメインかどうか判定
def is_same_domain(url: str) -> bool:
    return urlparse(url).netloc == DOMAIN

# URLの正規化（相対→絶対、フラグメント削除、拡張子除外など）
def normalize_url(base: str, href: str | None) -> str | None:
    if not href:
        return None
    # スキーム除外
    if href.startswith(("javascript:", "mailto:", "tel:")):
        return None
    abs_url = urljoin(base, href)
    abs_url, _ = urldefrag(abs_url)

    # http を https に統一（可能な場合）
    if abs_url.startswith("http://"):
        abs_url = abs_url.replace("http://", "https://", 1)

    # 拡張子で除外（クエリは無視してパスで判定）
    path = urlparse(abs_url).path.lower()
    for ext in EXCLUDE_EXTS:
        if path.endswith(ext):
            return None

    return abs_url

# コメント内リンクを除外して <a href> を抽出
def extract_links_excluding_comments(soup: BeautifulSoup, base_url: str) -> set[str]:
    # コメントノードを取り除く（コメント内の<a>は対象外）
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        comment.extract()

    links: set[str] = set()
    for a in soup.find_all("a", href=True):
        url = normalize_url(base_url, a["href"])
        if url:
            links.add(url)
    return links

# <title>抽出（空ならNone）
def extract_title(soup: BeautifulSoup) -> str | None:
    if soup.title and soup.title.string:
        return soup.title.string.strip()
    og = soup.find("meta", property="og:title")
    if og and og.get("content"):
        return og["content"].strip()
    h1 = soup.find("h1")
    if h1 and h1.get_text(strip=True):
        return h1.get_text(strip=True)
    return None

def build_robot_parser(start_url: str) -> robotparser.RobotFileParser | None:
    if not RESPECT_ROBOTS:
        return None
    rp = robotparser.RobotFileParser()
    robots_url = urljoin(start_url, "/robots.txt")
    try:
        rp.set_url(robots_url)
        rp.read()
    except Exception:
        # robots取得に失敗した場合は None（必要なら厳格にブロックする実装へ変更可能）
        return None
    return rp

def can_fetch(url: str, rp: robotparser.RobotFileParser | None) -> bool:
    if rp is None or not RESPECT_ROBOTS:
        return True
    try:
        return rp.can_fetch("*", url)
    except Exception:
        return False

def crawl(start_url: str) -> dict[str, str | None]:
    site_map: dict[str, str | None] = {}  # key: URL, value: タイトル文字列（HTMLのみ）
    visited: set[str] = set()
    queue: deque[str] = deque([start_url])

    session = make_session()
    rp = build_robot_parser(start_url)

    pages_crawled = 0

    while queue:
        current = queue.popleft()

        # 正規化（http→https）
        if current.startswith("http://"):
            current = current.replace("http://", "https://", 1)

        if current in visited:
            continue
        if not is_same_domain(current):
            continue
        if not can_fetch(current, rp):
            # robots.txtで禁止ならスキップ
            visited.add(current)
            continue

        # アクセス前に待機（負荷軽減）
        polite_sleep()

        try:
            print(f"[info] fetching: {current}", flush=True)
            resp = session.get(current, timeout=CONNECT_READ_TIMEOUT)
            resp.raise_for_status()
        except requests.RequestException as e:
            print(f"[warn] fetch error: {current} -> {e}")
            visited.add(current)
            continue

        final_url = resp.url
        if not is_same_domain(final_url):
            visited.add(current)
            continue

        # Content-TypeでHTML以外を除外
        ctype = resp.headers.get("Content-Type", "").lower()
        if "text/html" not in ctype:
            visited.add(final_url)
            continue

        # 文字化け対策: バイト列をそのままBeautifulSoupへ渡す
        soup = BeautifulSoup(resp.content, "lxml")

        # タイトル抽出して辞書へ格納
        title = extract_title(soup)
        site_map[final_url] = title

        visited.add(final_url)
        pages_crawled += 1

        # MAX_PAGES到達で打ち切り
        if pages_crawled >= MAX_PAGES:
            print(f"[info] reached MAX_PAGES={MAX_PAGES}, stopping.", flush=True)
            break

        # リンク抽出（コメント除外、同一ドメイン、拡張子除外済み）
        links = extract_links_excluding_comments(soup, final_url)
        # 既訪問や他ドメインを除外してキューへ
        for link in links:
            if is_same_domain(link) and link not in visited:
                queue.append(link)

    return site_map

if __name__ == "__main__":
    result = crawl(START_URL)
    # 表示
    for url, title in result.items():
        print(f"{url}\t{title if title else ''}")